In [1]:
import pandas as pd
import numpy as np 
import missingno as msno
import os
import seaborn as sns
import matplotlib.pyplot as plt 
from scipy.stats import mannwhitneyu 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import statistics
# Сброс ограничений на число отображаемых столбцов, строк
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import sklearn as sk 
from sklearn.linear_model import LogisticRegression 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.neural_network import MLPClassifier 
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Datasets/CS

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Datasets/CS


In [3]:
train = pd.read_csv('train.csv')
#удвоение датасета реплейсом команд местами
train_1 = train[['map_id', 'team2_id', 'team1_id', 'map_name', 'who_win']]
train_1.rename(columns={'team2_id': 'team1_id', 'team1_id': 'team2_id'}, inplace=True)
train_1.loc[(train_1.who_win == 0), 'who_win'] = 2
train_1.loc[(train_1.who_win == 1), 'who_win'] = 0
train_1.loc[(train_1.who_win == 2), 'who_win'] = 1
train_2 = pd.concat([train,train_1])
train = train_2.copy()

players_feats = pd.read_csv('players_feats.csv') 
#заполнение Nan нулями
df_1 = players_feats.copy()
avg=round((df_1.p1_team_win_percent_after_first_kill.mean()+df_1.p2_team_win_percent_after_first_kill.mean()+df_1.p3_team_win_percent_after_first_kill.mean()+
      df_1.p4_team_win_percent_after_first_kill.mean()+df_1.p5_team_win_percent_after_first_kill.mean()),1)/5
df_1.p1_team_win_percent_after_first_kill = df_1.p1_team_win_percent_after_first_kill.fillna(avg)
df_1.p2_team_win_percent_after_first_kill = df_1.p2_team_win_percent_after_first_kill.fillna(avg)
df_1.p3_team_win_percent_after_first_kill = df_1.p3_team_win_percent_after_first_kill.fillna(avg)
df_1.p4_team_win_percent_after_first_kill = df_1.p4_team_win_percent_after_first_kill.fillna(avg)
df_1.p5_team_win_percent_after_first_kill = df_1.p5_team_win_percent_after_first_kill.fillna(avg)
players_feats = df_1.copy()
#заполнение 3-х столбцов с нейдостающей инфой
pf=players_feats.copy()
for i in range(1,6):
    ind = list(pf[(pf['p'+str(i)+'_rounds_played'] != 0) & (pf['p'+str(i)+'_kill_death'] == 0)].index)
    pf.loc[ind,'p'+str(i)+'_kill_death'] = pf.loc[ind,'p'+str(i)+'_kd_ratio']
    ind = list(pf[(pf['p'+str(i)+'_rounds_played'] != 0) & (pf['p'+str(i)+'_kill_round'] == 0)].index)
    pf.loc[ind,'p'+str(i)+'_kill_round'] = pf.loc[ind,'p'+str(i)+'_kills_per_round']
    ind = list(pf[(pf['p'+str(i)+'_rounds_played'] != 0) & (pf['p'+str(i)+'_kill_death_difference'] == 0)].index)
    pf.loc[ind,'p'+str(i)+'_kill_death_difference'] = pf.loc[ind,'p'+str(i)+'_total_kills'] - pf.loc[ind,'p'+str(i)+'_total_deaths']
players_feats = pf.copy()
#среднее фитов по игрокам команды
pf = players_feats.copy()
clmns = list(players_feats.columns)[1:25]
for i in range(len(clmns)):
  clmns[i] = clmns[i].replace('p1','',1)
for clm in clmns:
  for row in range(pf['p1_total_kills'].count()):
    vals = []
    for k in range(1,6):
      vals.append(pf.loc[row,'p'+str(k)+clm])
    avg = statistics.mean(vals)
    pf.loc[row,'p1'+clm] = avg
pf = pf.drop(columns = list(players_feats.columns)[25:125])
pf = pf.drop(columns = ['p1_id'])
players_feats = pf.copy()
#нормализация по раундам, процентам
#делим на кол-во раундов
clmns = ['p1_rounds_with_kills','p1_kill_death_difference',	'p1_total_opening_kills',	'p1_total_opening_deaths']
pf = players_feats.copy()
rows = list(pf[pf.p1_rounds_played != 0].p1_rounds_played.index)
for clm in clmns:
  for row in rows:
    pf.loc[row,clm] = pf.loc[row,clm]/pf.loc[row,'p1_rounds_played']
#делим на 100
clmns = ['p1_headshots','p1_damage_per_round', 'p1_grenade_damage_per_round', 'p1_team_win_percent_after_first_kill','p1_first_kill_in_won_rounds']
row = list(i for i in range (pf['p1_total_kills'].count()))
for clm in clmns:
  pf.loc[row,clm] = pf.loc[row,clm]/100
players_feats = pf.copy()
#делим все на max
clmns = list(players_feats.columns)[0:24]
pf = players_feats.copy()
row = list(i for i in range (pf['p1_total_kills'].count()))
for clm in clmns:
  max = pf[clm].max()
  pf.loc[row,clm] = round(pf.loc[row,clm]/max,6)
pf = pf.drop(columns = ['p1_kill_death','p1_kill_round'])
players_feats = pf.copy()

print(players_feats.shape)
players_feats.head(5)

(1486, 25)


,p1_total_kills,p1_headshots,p1_total_deaths,p1_kd_ratio,p1_damage_per_round,p1_grenade_damage_per_round,p1_maps_played,p1_rounds_played,p1_kills_per_round,p1_assists_per_round,p1_deaths_per_round,p1_saved_by_teammate_per_round,p1_saved_teammates_per_round,p1_rating,p1_rounds_with_kills,p1_kill_death_difference,p1_total_opening_kills,p1_total_opening_deaths,p1_opening_kill_ratio,p1_opening_kill_rating,p1_team_win_percent_after_first_kill,p1_first_kill_in_won_rounds,team_id,map_name,map_id
0,0.128440,0.769599,0.157670,0.525180,0.774746,0.568548,0.146341,0.147005,0.677852,0.757895,0.882353,0.571429,0.478873,0.688126,0.800691,-0.390166,0.671415,0.839506,0.185322,0.713608,0.719398,0.654661,6665,Ancient,635
1,0.169069,0.823006,0.164773,0.489209,0.733160,0.546371,0.170732,0.157895,0.662192,0.505263,0.956522,0.328571,0.492958,0.599428,0.745471,0.139715,0.669761,0.597701,0.189832,0.738924,0.547240,0.759534,7532,Ancient,635
2,0.182176,0.713797,0.196023,0.597122,0.764782,0.508065,0.195122,0.199637,0.711409,0.673684,0.820972,0.642857,0.563380,0.729614,0.836000,0.000000,0.812238,0.727273,0.234522,0.824367,0.758892,0.825212,6665,Dust2,583
3,0.571429,0.876412,0.572443,0.646283,0.808967,0.518145,0.560976,0.577132,0.762864,0.652632,0.813299,0.685714,0.690141,0.789700,0.800579,0.191119,0.757378,0.742138,0.224682,0.819620,0.846171,0.854873,7532,Dust2,583
4,0.209699,0.846286,0.177557,0.768585,0.831276,0.447581,0.219512,0.204174,0.798658,0.652632,0.710997,0.614286,0.619718,0.855508,0.937884,0.756322,0.897778,0.640000,0.357114,0.892405,0.853589,0.826271,4608,Dust2,439


In [4]:
# смерживаем средние показатели по командам с размеченными играми
df = train.merge(players_feats ,left_on=['map_id', 'team1_id'], right_on = ['map_id','team_id'], how = 'left')
df = df.merge(players_feats,left_on=['map_id', 'team2_id'], right_on = ['map_id','team_id'], how = 'left')
df = df.drop(labels=['map_id','team1_id','team2_id','map_name_x','team_id_x','map_name_y','team_id_y','map_name'],axis=1)
df.head(5)

,who_win,p1_total_kills_x,p1_headshots_x,p1_total_deaths_x,p1_kd_ratio_x,p1_damage_per_round_x,p1_grenade_damage_per_round_x,p1_maps_played_x,p1_rounds_played_x,p1_kills_per_round_x,p1_assists_per_round_x,p1_deaths_per_round_x,p1_saved_by_teammate_per_round_x,p1_saved_teammates_per_round_x,p1_rating_x,p1_rounds_with_kills_x,p1_kill_death_difference_x,p1_total_opening_kills_x,p1_total_opening_deaths_x,p1_opening_kill_ratio_x,p1_opening_kill_rating_x,p1_team_win_percent_after_first_kill_x,p1_first_kill_in_won_rounds_x,p1_total_kills_y,p1_headshots_y,p1_total_deaths_y,p1_kd_ratio_y,p1_damage_per_round_y,p1_grenade_damage_per_round_y,p1_maps_played_y,p1_rounds_played_y,p1_kills_per_round_y,p1_assists_per_round_y,p1_deaths_per_round_y,p1_saved_by_teammate_per_round_y,p1_saved_teammates_per_round_y,p1_rating_y,p1_rounds_with_kills_y,p1_kill_death_difference_y,p1_total_opening_kills_y,p1_total_opening_deaths_y,p1_opening_kill_ratio_y,p1_opening_kill_rating_y,p1_team_win_percent_after_first_kill_y,p1_first_kill_in_won_rounds_y
0,0,0.352556,0.681274,0.363636,0.645084,0.821096,0.576613,0.341463,0.359347,0.771812,0.726316,0.820972,0.628571,0.619718,0.776824,0.875111,0.159613,0.804390,0.808081,0.217712,0.799051,0.815841,0.751059,0.152031,0.771311,0.174716,0.579137,0.763483,0.441532,0.170732,0.166062,0.713647,0.642105,0.861893,0.485714,0.478873,0.702432,0.825180,-0.159412,0.764187,0.786885,0.241082,0.792722,0.703469,0.756356
1,0,0.322412,0.772338,0.375000,0.561151,0.775179,0.816532,0.365854,0.361162,0.693512,0.757895,0.846547,0.742857,0.718310,0.726753,0.817206,-0.207676,0.878431,0.904523,0.225092,0.837025,0.775693,0.863347,0.705111,0.824375,0.690341,0.676259,0.798571,0.659274,0.731707,0.719601,0.762864,0.621053,0.780051,0.685714,0.647887,0.802575,0.598134,0.159412,0.578039,0.504414,0.284543,0.860759,0.796422,0.817797
2,1,0.530799,0.808285,0.508523,0.679856,0.830409,0.445565,0.512195,0.515426,0.796421,0.610526,0.805627,0.571429,0.577465,0.801144,0.610113,0.094160,0.588164,0.464789,0.248052,0.854430,0.810604,0.852754,0.300131,0.648408,0.299716,0.649880,0.810266,0.475806,0.317073,0.305808,0.765101,0.757895,0.805627,0.585714,0.605634,0.779685,0.626184,-0.028855,0.599407,0.522255,0.265683,0.830696,0.704560,0.798729
3,0,0.458716,0.815817,0.434659,0.684652,0.813732,0.383065,0.414634,0.460073,0.769575,0.589474,0.772379,0.500000,0.521127,0.798283,0.832442,0.421955,0.781520,0.583826,0.275523,0.833861,0.809295,0.743644,0.395806,0.794933,0.417614,0.610312,0.787524,0.455645,0.365854,0.406534,0.740492,0.578947,0.831202,0.571429,0.591549,0.733906,0.825393,0.086823,0.693681,0.732143,0.196802,0.770570,0.772420,0.717161
4,1,0.331586,0.768230,0.343750,0.631894,0.804635,0.346774,0.317073,0.337568,0.769575,0.663158,0.836317,0.657143,0.633803,0.769671,0.556860,0.000000,0.459471,0.430108,0.234522,0.806962,0.826533,0.768008,0.224115,0.849709,0.224432,0.663070,0.831709,0.364919,0.219512,0.220508,0.791946,0.610526,0.823529,0.685714,0.647887,0.788269,0.796708,0.160068,0.671415,0.757202,0.241902,0.784810,0.854899,0.792373


In [5]:
#models training
y_df = df.iloc[:,0] 
x_df = df.iloc[:,1:] 
#x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.0, train_size=0.975, random_state=100)
x_train=x_df.copy()
y_train=y_df.copy()
print(x_train.shape)
#print(x_test.shape)
def per_0_1(sampl):
  print('0:',round(sampl[sampl ==0].count()/sampl.count(),2),'1:', round(sampl[sampl ==1].count()/sampl.count(),2))
per_0_1(y_train)
#per_0_1(y_test)

LR_model = LogisticRegression(penalty='l1', solver='liblinear', multi_class='ovr', max_iter=1000)
LR_model.fit(x_train, y_train) 
LR_prediction = LR_model.predict(x_train)
print('LR train: acc=',accuracy_score(LR_prediction, y_train), '\troc_auc =', roc_auc_score(LR_prediction, y_train))
scores = cross_val_score(LR_model, x_train, y_train, cv=5, scoring='roc_auc')
print('cross-val_roc_auc:',scores.mean())

SVC_model = SGDClassifier(max_iter = 10000, random_state=1, alpha = 0.1)
SVC_model.fit(x_train, y_train)
SVC_prediction = SVC_model.predict(x_train)
print('SVC train: acc=',accuracy_score(SVC_prediction, y_train), '\troc_auc =', roc_auc_score(SVC_prediction, y_train))
scores = cross_val_score(SVC_model, x_train, y_train, cv=5, scoring='roc_auc')
print('cross-val_roc_auc:',scores.mean())

RF_model = RandomForestClassifier(n_estimators=400, max_depth=10, criterion = 'entropy',min_samples_split=4, random_state=1) 
RF_model.fit(x_train, y_train)
RF_prediction = RF_model.predict(x_train)
print('RF train: acc=',accuracy_score(RF_prediction, y_train), '\troc_auc =', roc_auc_score(RF_prediction, y_train))
scores = cross_val_score(RF_model, x_train, y_train, cv=5, scoring='roc_auc')
print('cross-val_roc_auc:',scores.mean())

NN_model = MLPClassifier(solver='sgd', max_iter=10000, alpha=1e-5, hidden_layer_sizes=(57, ),random_state=1)
NN_model.fit(x_train, y_train)
NN_prediction = NN_model.predict(x_train)
print('NN train: acc=',accuracy_score(NN_prediction, y_train), '\troc_auc =', roc_auc_score(NN_prediction, y_train))
scores = cross_val_score(NN_model, x_train, y_train, cv=5, scoring='roc_auc')
print('cross-val_roc_auc:',scores.mean())


(1426, 44)
0: 0.5 1: 0.5
LR train: acc= 0.6437587657784011 	roc_auc = 0.6437587657784012
cross-val_roc_auc: 0.6691800763011243
SVC train: acc= 0.6276297335203366 	roc_auc = 0.6328241430700446
cross-val_roc_auc: 0.6648315045537542
RF train: acc= 0.9698457223001402 	roc_auc = 0.9698466465237782
cross-val_roc_auc: 0.6348238386256706
NN train: acc= 0.6276297335203366 	roc_auc = 0.6276721763085399
cross-val_roc_auc: 0.6686159108300348


In [6]:
#смерживаем средние показатели по командам с неразмеченными играми
test = pd.read_csv('test.csv') 
df_test = test.merge(players_feats ,left_on=['map_id', 'team1_id'], right_on = ['map_id','team_id'], how = 'left')
df_test = df_test.merge(players_feats,left_on=['map_id', 'team2_id'], right_on = ['map_id','team_id'], how = 'left')
df_test = df_test.drop(labels=['map_id','team1_id','team2_id','map_name_x','team_id_x','map_name_y','team_id_y','map_name','index'],axis=1)
x_test = df_test.copy()
print(x_test.shape)
x_test.head(5)

(30, 44)


,p1_total_kills_x,p1_headshots_x,p1_total_deaths_x,p1_kd_ratio_x,p1_damage_per_round_x,p1_grenade_damage_per_round_x,p1_maps_played_x,p1_rounds_played_x,p1_kills_per_round_x,p1_assists_per_round_x,p1_deaths_per_round_x,p1_saved_by_teammate_per_round_x,p1_saved_teammates_per_round_x,p1_rating_x,p1_rounds_with_kills_x,p1_kill_death_difference_x,p1_total_opening_kills_x,p1_total_opening_deaths_x,p1_opening_kill_ratio_x,p1_opening_kill_rating_x,p1_team_win_percent_after_first_kill_x,p1_first_kill_in_won_rounds_x,p1_total_kills_y,p1_headshots_y,p1_total_deaths_y,p1_kd_ratio_y,p1_damage_per_round_y,p1_grenade_damage_per_round_y,p1_maps_played_y,p1_rounds_played_y,p1_kills_per_round_y,p1_assists_per_round_y,p1_deaths_per_round_y,p1_saved_by_teammate_per_round_y,p1_saved_teammates_per_round_y,p1_rating_y,p1_rounds_with_kills_y,p1_kill_death_difference_y,p1_total_opening_kills_y,p1_total_opening_deaths_y,p1_opening_kill_ratio_y,p1_opening_kill_rating_y,p1_team_win_percent_after_first_kill_y,p1_first_kill_in_won_rounds_y
0,0.190039,0.711058,0.176136,0.695444,0.845571,0.455645,0.195122,0.176044,0.821029,0.642105,0.818414,0.757143,0.788732,0.828326,0.858227,0.526306,0.921094,0.618557,0.308323,0.909810,0.824351,0.907839,0.258191,0.765491,0.238636,0.709832,0.818497,0.520161,0.292683,0.259528,0.771812,0.663158,0.754476,0.457143,0.464789,0.814020,0.900308,0.476007,0.923615,0.615385,0.398114,0.903481,0.799913,0.819915
1,0.326343,0.730914,0.316761,0.659472,0.848603,0.629032,0.317073,0.310345,0.807606,0.768421,0.846547,0.828571,0.845070,0.802575,0.826480,0.127949,0.726946,0.725146,0.212382,0.805380,0.755400,0.754237,0.264744,0.638138,0.242898,0.724221,0.836041,0.806452,0.243902,0.262250,0.787472,0.652632,0.759591,0.657143,0.661972,0.834049,0.937855,0.521537,0.833378,0.719723,0.281673,0.837025,0.822169,0.758475
2,0.356488,0.730229,0.404830,0.577938,0.775179,0.324597,0.414634,0.382940,0.722595,0.642105,0.864450,0.600000,0.577465,0.711016,0.743204,-0.322602,0.644367,0.758294,0.187782,0.753165,0.706961,0.711864,0.260813,0.750770,0.259943,0.684652,0.827160,0.326613,0.268293,0.261343,0.787472,0.621053,0.795396,0.642857,0.788732,0.801144,0.732722,0.185704,0.647436,0.666667,0.223862,0.810127,0.846607,0.780720
3,0.373526,0.699418,0.401989,0.605516,0.776911,0.348790,0.390244,0.397459,0.727069,0.673684,0.831202,0.500000,0.492958,0.739628,0.884018,0.022201,0.815947,0.748858,0.238212,0.819620,0.808422,0.817797,0.283093,0.824718,0.308239,0.651079,0.829976,0.250000,0.268293,0.292196,0.785235,0.568421,0.841432,0.671429,0.690141,0.765379,0.703453,0.000000,0.530817,0.670807,0.141041,0.587025,0.625573,0.536017
4,0.167759,0.774392,0.166193,0.663070,0.822179,0.312500,0.170732,0.169691,0.769575,0.631579,0.800512,0.471429,0.478873,0.772532,0.911059,0.286004,0.830934,0.727273,0.263633,0.832278,0.870172,0.792373,0.222805,0.724409,0.231534,0.624700,0.792723,0.336694,0.219512,0.229583,0.742729,0.610526,0.818414,0.900000,0.718310,0.765379,0.681043,0.057653,0.614168,0.664032,0.233702,0.835443,0.811477,0.818856


In [7]:
# вычисление предсказаний, запись в файлы
LR_prediction = list(LR_model.predict(x_test))
pred_list=LR_prediction
print(pred_list)
pred_df = pd.DataFrame({'who_won': pred_list})
pred_df.to_csv('prediction_1.csv',index=False)

SVC_prediction = list(SVC_model.predict(x_test))
pred_list=SVC_prediction
print(pred_list)
pred_df = pd.DataFrame({'who_won': pred_list})
pred_df.to_csv('prediction_2.csv',index=False)

NN_prediction = list(NN_model.predict(x_test))
pred_list=NN_prediction
print(pred_list)
pred_df = pd.DataFrame({'who_won': pred_list})
pred_df.to_csv('prediction_3.csv',index=False)
#print(pred_df)

[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1]
[0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1]
[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1]


In [ ]:
'''#cv=5
(1426, 44)
0: 0.5 1: 0.5
LR train: acc= 0.6437587657784011 	roc_auc = 0.6437587657784012
cross-val_roc_auc: 0.6691900633592744
SVC train: acc= 0.6276297335203366 	roc_auc = 0.6328241430700446
cross-val_roc_auc: 0.6648315045537542
RF train: acc= 0.9698457223001402 	roc_auc = 0.9698466465237782
cross-val_roc_auc: 0.6348238386256706
NN train: acc= 0.6276297335203366 	roc_auc = 0.6276721763085399
cross-val_roc_auc: 0.6686159108300348
#cv=45
(1426, 44)
0: 0.5 1: 0.5
LR train: acc= 0.6437587657784011 	roc_auc = 0.6437587657784012
cross-val_roc_auc: 0.6742534722222222
SVC train: acc= 0.6276297335203366 	roc_auc = 0.6328241430700446
cross-val_roc_auc: 0.6648842592592593
RF train: acc= 0.9698457223001402 	roc_auc = 0.9698466465237782
cross-val_roc_auc: 0.6355150462962963
NN train: acc= 0.6276297335203366 	roc_auc = 0.6276721763085399
cross-val_roc_auc: 0.6707928240740741
#удалил повторные рейты cv=5
(1426, 44)
0: 0.5 1: 0.5
LR train: acc= 0.635343618513324 	roc_auc = 0.635343618513324
cross-val_roc_auc: 0.6663826668751321
SVC train: acc= 0.6640953716690042 	roc_auc = 0.6640956944575582
cross-val_roc_auc: 0.6516337105227089
RF train: acc= 0.8892005610098177 	roc_auc = 0.8892128107830582
cross-val_roc_auc: 0.6403163762269447
NN train: acc= 0.6297335203366059 	roc_auc = 0.6362562056719273
cross-val_roc_auc: 0.6622401040307078
#поделил "последние" 4 фита на раунды
(1426, 48)
0: 0.5 1: 0.5
LR train: acc= 0.6339410939691444 	roc_auc = 0.6339410939691446
cross-val_roc_auc: 0.6665593344899954
SVC train: acc= 0.6605890603085554 	roc_auc = 0.6605890603085554
cross-val_roc_auc: 0.6508083662619267
RF train: acc= 0.8863955119214586 	roc_auc = 0.8864441624615143
cross-val_roc_auc: 0.6395519874590099
NN train: acc= 0.6262272089761571 	roc_auc = 0.6269285544742464
cross-val_roc_auc: 0.6564802576523251
#/max adam
(1426, 48)
0: 0.5 1: 0.5
LR train: acc= 0.6493688639551192 	roc_auc = 0.6493688639551192
cross-val_roc_auc: 0.6621010428554996
SVC train: acc= 0.6311360448807855 	roc_auc = 0.6311360448807856
cross-val_roc_auc: 0.6413212120293771
RF train: acc= 0.8772791023842917 	roc_auc = 0.8772798445220786
cross-val_roc_auc: 0.6120877841748521
NN train: acc= 0.6332398316970547 	roc_auc = 0.6424177053371685
cross-val_roc_auc: 0.6568956503951087
#mean lbfs
(1426, 48)
0: 0.5 1: 0.5
LR train: acc= 0.6311360448807855 	roc_auc = 0.6311360448807856
cross-val_roc_auc: 0.654913253790433
SVC train: acc= 0.6086956521739131 	roc_auc = 0.6086956521739131
cross-val_roc_auc: 0.6318815135421065
RF train: acc= 0.6136044880785414 	roc_auc = 0.6136691072272084
cross-val_roc_auc: 0.595470042613744
NN train: acc= 0.6339410939691444 	roc_auc = 0.6339421478661985
cross-val_roc_auc: 0.659692577687259
#stat.mean
(1426, 48)
0: 0.5 1: 0.5
LR train: acc= 0.6311360448807855 	roc_auc = 0.6311360448807856
cross-val_roc_auc: 0.654913253790433
SVC train: acc= 0.6086956521739131 	roc_auc = 0.6086956521739131
cross-val_roc_auc: 0.6318815135421065
RF train: acc= 0.6136044880785414 	roc_auc = 0.6136691072272084
cross-val_roc_auc: 0.595470042613744
NN train: acc= 0.6311360448807855 	roc_auc = 0.6356453814991383
cross-val_roc_auc: 0.6216326567158833
(713, 48)
0: 0.54 1: 0.46
LR train: acc= 0.6521739130434783 	roc_auc = 0.6504280793561422
cross-val_roc_auc: 0.6297686462721428
SVC train: acc= 0.5862552594670407 	roc_auc = 0.6466025307678974
cross-val_roc_auc: 0.6081835181135882
RF train: acc= 0.6269284712482468 	roc_auc = 0.6849557522123895
cross-val_roc_auc: 0.5937800195142853
NN train: acc= 0.5708274894810659 	roc_auc = 0.6842032041706939
cross-val_roc_auc: 0.6112433021523931
#средние значения
LR train: acc= 0.6479663394109397 	roc_auc = 0.6463835534213686
cross-val_roc_auc: 0.625062163943283
SVC train: acc= 0.603085553997195 	roc_auc = 0.632427843803056
cross-val_roc_auc: 0.5659533473519487
RF train: acc= 0.635343618513324 	roc_auc = 0.7113330200501253
cross-val_roc_auc: 0.599971008012966
NN train: acc= 0.6044880785413744 	roc_auc = 0.7034161490683231
cross-val_roc_auc: 0.6299813007505316
#заменил на mean 7 столбцов
(713, 240)
0: 0.54 1: 0.46
LR train: acc= 0.7166900420757363 	roc_auc = 0.7150920235554628
cross-val_roc_auc: 0.5552697651998351
SVC train: acc= 0.4908835904628331 	roc_auc = 0.6851179673321234
cross-val_roc_auc: 0.5734624815743696
RF train: acc= 0.9214586255259467 	roc_auc = 0.9366515837104072
cross-val_roc_auc: 0.5489819271637454
NN train: acc= 0.697054698457223 	roc_auc = 0.7333487713656541
cross-val_roc_auc: 0.5515502446271677
#посчитал 3 столбца нулевых
(713, 240)
0: 0.54 1: 0.46
LR train: acc= 0.7489481065918654 	roc_auc = 0.7476421126670503
cross-val_roc_auc: 0.6211145032124052
SVC train: acc= 0.6325385694249649 	roc_auc = 0.6774249433446331
cross-val_roc_auc: 0.614027883538373
RF train: acc= 0.9074333800841514 	roc_auc = 0.9269911504424779
cross-val_roc_auc: 0.5805004785424366
NN train: acc= 0.7251051893408135 	roc_auc = 0.7619623655913978
cross-val_roc_auc: 0.5795050171273948
#base
(713, 240)
(18, 240)
0: 0.54 1: 0.46
0: 0.28 1: 0.72
LR train: acc= 0.7517531556802244 	roc_auc = 0.7502030093145451
cross-val_roc_auc: 0.6201618428191855
SVC train: acc= 0.49228611500701264 	roc_auc = 0.7015119916579771
cross-val_roc_auc: 0.5957154267643778
RF train: acc= 0.9046283309957924 	roc_auc = 0.9251101321585903
cross-val_roc_auc: 0.5800975714262429
NN train: acc= 0.6942496493688639 	roc_auc = 0.7541749502982107
cross-val_roc_auc: 0.56597467800265

(1426, 240)
(18, 240)
0: 0.5 1: 0.5
0: 0.28 1: 0.72
LR train: acc= 0.6942496493688639 	roc_auc = 0.6942496493688639
cross-val_roc_auc: 0.6306214911848714
SVC train: acc= 0.5995792426367461 	roc_auc = 0.6436749313171221
cross-val_roc_auc: 0.5872722175883802
RF train: acc= 0.8758765778401122 	roc_auc = 0.8759128108278251
cross-val_roc_auc: 0.5914995671126864
NN train: acc= 0.6269284712482468 	roc_auc = 0.7215783636032732
cross-val_roc_auc: 0.5846441748933623'''

SyntaxError: ignored